# Imbalanced Data

In [1]:
import pandas as pd
import numpy as np


pd.set_option('display.max_columns', None)


df = pd.read_csv('../data/processed/processed_data.csv')
df.head()

,has_null,wave,age,age_o,d_age,d_d_age,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match,person_id,nulls_byRow,gender_female,gender_male,race_Asian/Pacific Islander/Asian-American,race_Black/African American,race_European/Caucasian-American,race_Latino/Hispanic American,race_Other,race_o_?,race_o_Asian/Pacific Islander/Asian-American,race_o_Black/African American,race_o_European/Caucasian-American,race_o_Latino/Hispanic American,race_o_Other,field_anthropology,field_architecture,field_art,field_biology,field_business,field_chemistry,field_communications,field_economics,field_education,field_engineering,field_environmental science,field_finance,field_health,field_history,field_law,field_literature,field_mathematics,field_medicine,field_other/job,field_philosophy,field_physics,field_political science,field_psychology,field_religion,field_sociology
0,0.0,1.0,21.0,27.0,6.0,2.0,0.0,2.0,4.0,1.0,1.0,35.0,20.0,20.0,20.0,0.0,5.0,2.0,1.0,1.0,1.0,0.0,0.0,6.0,8.0,8.0,8.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,15.0,20.0,20.0,15.0,15.0,15.0,0.0,1.0,1.0,0.0,0.0,0.0,6.0,8.0,8.0,8.0,7.0,1.0,1.0,1.0,1.0,1.0,6.0,9.0,7.0,7.0,6.0,5.0,1.0,2.0,1.0,1.0,1.0,0.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.14,0.0,3.0,4.0,0.0,0.0,1.0,7.0,6.00,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,21.0,22.0,1.0,0.0,0.0,2.0,4.0,1.0,1.0,60.0,0.0,0.0,40.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,1.0,1.0,2.0,1.0,1.0,0.0,15.0,20.0,20.0,15.0,15.0,15.0,0.0,1.0,1.0,0.0,0.0,0.0,6.0,8.0,8.0,8.0,7.0,1.0,1.0,1.0,1.0,1.0,7.0,8.0,7.0,8.0,5.0,6.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.54,1.0,3.0,4.0,0.0,0.0,1.0,7.0,5.00,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,21.0,22.0,1.0,0.0,1.0,2.0,4.0,1.0,1.0,19.0,18.0,19.0,18.0,14.0,12.0,1.0,1.0,1.0,1.0,0.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0,2.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,0.0,1.0,1.0,0.0,0.0,0.0,6.0,8.0,8.0,8.0,7.0,1.0,1.0,1.0,1.0,1.0,5.0,8.0,9.

In [2]:
import plotly.express as px

fig = px.bar(x=df.match.value_counts().index, y=df.match.value_counts().values/len(df)*100, text_auto='.2f')
fig.show()

In [18]:
lista = list(df.columns)

In [19]:
lista.remove('wave')
lista.remove('decision')
lista.remove('decision_o')
lista.remove('match')
lista.remove('nulls_byRow')

In [23]:
y = df['match']
X = df[lista]

from sklearn.model_selection import train_test_split

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=12)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=12) 

X_train.to_csv('../data/modelling/train/X_train.csv', index=False)
y_train.to_csv('../data/modelling/train/y_train.csv', index=False)

X_test.to_csv('../data/modelling/test/X_test.csv', index=False)
y_test.to_csv('../data/modelling/test/y_test.csv', index=False)

X_val.to_csv('../data/modelling/validation/X_val.csv', index=False)
y_val.to_csv('../data/modelling/validation/y_val.csv', index=False)

### Dealing with Inbalanced data

1. Change the performance metric
2. Oversample minority class
3. Undersample majority class
4. Generate synthetic samples

#### 1. Change the performance metric

For an imbalanced class dataset <b>F1 score</b> is a more appropriate metric. It is the harmonic mean of precision and recall. So, if the classifier predicts the minority class but the prediction is erroneous and false-positive increases, the precision metric will be low and so as F1 score. Also, if the classifier identifies the minority class poorly, i.e. more of this class wrongfully predicted as the majority class then false negatives will increase, so recall and F1 score will low. F1 score only increases if both the number and quality of prediction improves.

#### 2. Oversample minority class

In [24]:
train = pd.concat([X_train, y_train], axis=1)

In [26]:
from sklearn.utils import resample

#create two different dataframe of majority and minority class 
df_majority = train[(train['match']==0)] 
df_minority = train[(train['match']==1)] 

# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= len(df_majority), # to match majority class
                                 random_state=12)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

# check new class counts
df_upsampled.match.value_counts()

df_upsampled.to_csv('../data/modelling/train/upsampled_train.csv', index=False)

#### 3. Undersample majority class

In [27]:
# downsample majority
df_majority_downsampled = resample(df_majority,
                                replace = False, # sample without replacement
                                n_samples = len(df_minority), # match minority n
                                random_state = 12) # reproducible results

# combine minority and downsampled majority
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# checking counts
df_downsampled.match.value_counts()

df_downsampled.to_csv('../data/modelling/train/downsampled_train.csv', index=False)

#### 4. Synthetic Minority Oversampling Technique or SMOTE

Synthetic Minority Oversampling Technique or SMOTE is another technique to oversample the minority class. Simply adding duplicate records of minority class often don’t add any new information to the model. In SMOTE new instances are synthesized from the existing data. If we explain it in simple words, SMOTE looks into minority class instances and use k nearest neighbor to select a random nearest neighbor, and a synthetic instance is created randomly in feature space.

In [29]:
from imblearn.over_sampling import SMOTENC

smote_nc = SMOTENC(categorical_features=[2, 5, 6, 10], random_state=12)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)

df_oversampled_sm = pd.concat([pd.DataFrame(y_resampled), pd.DataFrame(X_resampled)], axis=1)

df_oversampled_sm.to_csv('../data/modelling/train/oversampled_sm_train.csv', index=False)

https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
https://www.analyticsvidhya.com/blog/2021/06/5-techniques-to-handle-imbalanced-data-for-a-classification-problem/